This cell sets up the Google Colab environment to handle asynchronous programming and enables the display of rich HTML content.

1. **`import nest_asyncio`**: Imports the `nest_asyncio` library, which allows for the smooth handling of asynchronous tasks in environments like Google Colab, where nested event loops might otherwise cause issues.

2. **`nest_asyncio.apply()`**: Applies a patch that enables running asynchronous code without conflicts. This is particularly useful when working with `asyncio` in notebooks, allowing for proper execution of asynchronous tasks.

3. **`from IPython.display import display, HTML`**: Imports functions from IPython’s display module that allow for the rendering of rich media content (like HTML) directly within the notebook. This enables the output of formatted HTML, which can be used for interactive content or visually enriched outputs.

This setup ensures that you can effectively run and display asynchronous code alongside HTML content in the same notebook.

In [3]:
import nest_asyncio

nest_asyncio.apply()

from IPython.display import display, HTML

Ensure you have Ollama installed

In [4]:
!which ollama

/usr/local/bin/ollama


To install Ollama: https://ollama.com/download

Ensure you have Mistral in Ollama

In [6]:
!ollama list

NAME          	ID          	SIZE  	MODIFIED     
mistral:latest	f974a74358d6	4.1 GB	27 hours ago	
llama3:latest 	365c0bd3c000	4.7 GB	2 months ago	


To install Mistral on Ollama run the following cell

In [7]:
!ollama pull mistral

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         
pulling 491dfa501e59... 100% ▕████████████████▏  801 B                         
pulling ed11eda7790d... 100% ▕████████████████▏   30 B                         
pulling 42347cd80dc8... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [8]:
%pip install ollama --quiet

Note: you may need to restart the kernel to use updated packages.


In [22]:
import json
import ollama
import asyncio
import requests

# Set your Exa API key
EXA_API_KEY = "32d520b3-b8ee-4fad-88a6-18fec4b50ad6"

# Web search using Exa AI
def search_web(query: str) -> str:
  headers = {
    'accept': 'application/json',
    'content-type': 'application/json',
    'x-api-key': EXA_API_KEY,
  }
  data = {
    "query": query,
    "type": "neural",
    "useAutoprompt": True,
    "numResults": 10,
    "contents": {
      "text": True
    }
  }
  # Correct Exa AI search endpoint
  response = requests.post('https://api.exa.ai/search', headers=headers, json=data)
  
  # Check if the response is valid JSON
  try:
    response.raise_for_status()  # Raise an error for bad status codes
    response_json = response.json()
  except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
    return json.dumps({'error': 'HTTP error occurred'})
  except json.JSONDecodeError:
    print("Failed to decode JSON. Response content:")
    print(response.text)
    return json.dumps({'error': 'Failed to decode JSON response'})

  return json.dumps(response_json)

async def run(model: str):
  client = ollama.AsyncClient()
  # Initialize conversation with a user query
  messages = [{'role': 'user', 'content': 'Search the web for "current weather in New York"'}]

  # First API call: Send the query and function description to the model
  response = await client.chat(
    model=model,
    messages=messages,
    tools=[
      {
        'type': 'function',
        'function': {
          'name': 'search_web',
          'description': 'Search the web for a given query',
          'parameters': {
            'type': 'object',
            'properties': {
              'query': {
                'type': 'string',
                'description': 'The search query',
              },
            },
            'required': ['query'],
          },
        },
      },
    ],
  )

  # Add the model's response to the conversation history
  messages.append(response['message'])

  # Check if the model decided to use the provided function
  if not response['message'].get('tool_calls'):
    print("The model didn't use the function. Its response was:")
    print(response['message']['content'])
    return

  # Process function calls made by the model
  if response['message'].get('tool_calls'):
    available_functions = {
      'search_web': search_web,
    }
    for tool in response['message']['tool_calls']:
      function_to_call = available_functions[tool['function']['name']]
      function_response = function_to_call(tool['function']['arguments']['query'])
      # Add function response to the conversation
      messages.append(
        {
          'role': 'tool',
          'content': function_response,
        }
      )

  # Second API call: Get final response from the model
  final_response = await client.chat(model=model, messages=messages)
  print(final_response['message']['content'])


# Run the async function
asyncio.run(run('mistral'))

1. Title: Articles about humidity | Gothamist
     - Published Date: 2021-07-06T00:00:00.000Z
     - Author: Jen Chung
     - URL: https://gothamist.com/tags/humidity
     - Text: Latest Articles Tagged "humidity"... With a RealFeel™ of Lake of Fire... Humidity break? We'll take it.... And today is the NYC Triathlon!... Catch up on the most important headlines with a roundup of essential NYC stories, delivered to your inbox daily.

  2. Title: Articles about hot | Gothamist
     - Published Date: 2021-07-06T00:00:00.000Z
     - Author: Jen Chung
     - URL: https://gothamist.com/tags/hot
     - Text: Latest Articles Tagged "hot"... It's allegedly "cooling down" this week, if 80-degree weather strikes you as "cool."... It may not feel like it right now, but a sizzling heat wave will soon be on our doorstep.... Catch up on the most important headlines with a roundup of essential NYC stories, delivered to your inbox daily.

  3. Title: Articles about foggy | Gothamist
     - Published Dat

In [10]:
%pip install toolhouse --quiet

Note: you may need to restart the kernel to use updated packages.
